In [1]:
import numpy as np

from weldx import Q_, U_, GenericSeries, MathematicalExpression, Time, TimeSeries

In [2]:
class newTS(GenericSeries):

    # --- Constraints

    def preprocess_time(self, time, time_unit):
        time = Time(time)
        time_interp = Time(time, self._reference_time)
        return dict(t=time_interp.as_quantity().to(time_unit))

    _allowed_variables = ["t"]
    _required_dimensions = ["t"]
    _required_dimension_units = dict(t=U_("s"))
    _required_variables = ["t"]
    _evaluation_preprocessor = preprocess_time

    # --- Custom init

    def __init__(self, data, time=None, interpolation=None, reference_time=None):
        if time is not None:
            time = dict(t=Time(time, reference_time).as_quantity())

        self._reference_time = reference_time
        super().__init__(obj=data, coords=time, interpolation=interpolation)

    # --- extra properties and functions

    @property
    def reference_time(self):
        return self._reference_time

    @property
    def time(self):
        if self.is_expression:
            return None
        t = self._obj.coords["t"]
        return Time(Q_(t.data, t.attrs["units"]))  # There is probably a better way ;)

    def interp_time(self, time, time_unit="s"):
        return self(time=time, time_unit=time_unit)

## Discrete

### `__init__`

In [3]:
try:
    TimeSeries(Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "K"))
except Exception as e:
    print(e)

Cannot convert from 'kelvin' ([temperature]) to 'second' ([time])


In [4]:
try:
    newTS(Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "K"))
except Exception as e:
    print(e)

Cannot convert from 'kelvin' ([temperature]) to 'second' ([time])


In [5]:
ts_discrete = TimeSeries(
    Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "s"), reference_time="2000-01-01"
)

In [6]:
tsnew_discrete = newTS(
    Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "s"), reference_time="2000-01-01"
)

### attributes

In [7]:
np.allclose(ts_discrete.data, tsnew_discrete.data)

True

In [8]:
ts_discrete.data_array.equals(tsnew_discrete.data_array)

False

> Internal storage format differs (timedelta vs quantity). Also, the interpolation is yet not stored in the GS version

In [9]:
ts_discrete.data_array

Magnitude,[1 3 5]
Units,m


In [10]:
tsnew_discrete.data_array

Magnitude,[1 3 5]
Units,m


In [11]:
ts_discrete.is_discrete == tsnew_discrete.is_discrete == True

True

In [12]:
ts_discrete.is_expression == tsnew_discrete.is_expression == False

True

In [13]:
ts_discrete.units == tsnew_discrete.units

True

In [14]:
ts_discrete.interpolation == tsnew_discrete.interpolation

False

In [15]:
ts_discrete.interpolation

'step'

In [16]:
tsnew_discrete.interpolation

'linear'

In [17]:
np.all(ts_discrete.time == tsnew_discrete.time)

True

In [18]:
ts_discrete.reference_time == tsnew_discrete.reference_time

False

> GS version is correct, TS has a bug :)

In [19]:
ts_discrete.reference_time

In [20]:
tsnew_discrete.reference_time

'2000-01-01'

### interpolation

In [21]:
ts_discrete.interp_time("3s")

<TimeSeries>
Constant value:
	5
Units:
	m

In [22]:
tsnew_discrete.interp_time("3s")

<newTS>
Values:
[5.]
Dimensions:
	('t',)
Coordinates:
	Coordinates:
  * t        (t) float64 3.0
Units:
	m

In [23]:
d = tsnew_discrete.interp_time("3s")
d.data_array.data

Magnitude,[5.0]
Units,m


In [24]:
tsnew_discrete.data_array.sel(t=3).data

array(5) <Unit('meter')>

In [25]:
np.allclose(ts_discrete.interp_time("3s").data, tsnew_discrete.interp_time("3s").data)

True

In [26]:
np.allclose(
    ts_discrete.interp_time("2007").data, tsnew_discrete.interp_time("2007").data
)

False

> "Old TS" seems to be wrong here since it never got a proper support for absolute times. See below

In [27]:
ts_discrete.interp_time("2007").data

Magnitude,[1.0]
Units,m


In [28]:
tsnew_discrete.interp_time("2007").data

Magnitude,[5.0]
Units,m


In [29]:
np.allclose(
    ts_discrete.interp_time(Q_([1, 2, 3], "s")).data,
    tsnew_discrete.interp_time(Q_([1, 2, 3], "s")).data,
)

True

## Expression

### `__init__`

#### only time parameter allowed

In [30]:
try:
    TimeSeries(MathematicalExpression("a*t +b"))
except Exception as e:
    print(e)

The mathematical expression must have exactly 1 free variable that represents time.


In [31]:
try:
    newTS(MathematicalExpression("a*t +b"))
except Exception as e:
    print(e)

Cannot convert from 'second' to 'dimensionless'
Expression can not be evaluated due to a unit dimensionality error. Ensure that the expressions parameters and the expected variable units are compatible. The original exception was:
Cannot convert from 'second' ([time]) to 'dimensionless' (dimensionless)


#### units do not match cause t needs a time unit

In [32]:
try:
    TimeSeries(MathematicalExpression("a*t +b", parameters=dict(a=5, b=3)))
except Exception as e:
    print(e)

Expression can not be evaluated with "weldx.Quantity(1, "seconds")". Ensure that every parameter posses the correct unit.


In [33]:
### TODO: _required_dimension_units should automatically update the required units
try:
    newTS(MathematicalExpression("a*t +b", parameters=dict(a=5, b=3)))
except Exception as e:
    print(e)

Cannot convert from 'second' to 'dimensionless'
Expression can not be evaluated due to a unit dimensionality error. Ensure that the expressions parameters and the expected variable units are compatible. The original exception was:
Cannot convert from 'second' ([time]) to 'dimensionless' (dimensionless)


In [34]:
ts_expr = TimeSeries(
    MathematicalExpression("a*t +b", parameters=dict(a="5/s", b=3)),
    reference_time="2000-01-01",
);

In [35]:
tsnew_expr = newTS(
    MathematicalExpression("a*t +b", parameters=dict(a="5/s", b=3)),
    reference_time="2000-01-01",
);

### properties

In [36]:
ts_expr.data == tsnew_expr.data

True

In [37]:
ts_expr.data_array == tsnew_expr.data_array

True

In [38]:
ts_expr.is_discrete == tsnew_expr.is_discrete == False

True

In [39]:
ts_expr.is_expression == tsnew_expr.is_expression == True

True

In [40]:
ts_expr.units == tsnew_expr.units

True

In [41]:
# ts_expr.shape == ts_expr.shape

In [42]:
# tsnew_expr.shape

In [43]:
ts_expr.time == tsnew_expr.time

True

In [44]:
ts_expr.reference_time == tsnew_expr.reference_time

True

In [45]:
ts_expr.interpolation == tsnew_expr.interpolation

False

### interpolation

In [46]:
ts_expr.interp_time("3s")

<TimeSeries>
Constant value:
	18.0
Units:
	

In [47]:
tsnew_expr.interp_time("3s")

<newTS>
Values:
[18.]
Dimensions:
	('t',)
Coordinates:
	Coordinates:
  * t        (t) float64 3.0
Units:
	

> TODO: Should return a new `TimeSeries` and not a `GenericSeries`

In [48]:
np.allclose(ts_expr.interp_time("3s").data, tsnew_expr.interp_time("3s").data)

True

In [49]:
np.allclose(ts_expr.interp_time("2007").data, tsnew_expr.interp_time("2007").data)

True

In [50]:
np.allclose(
    ts_expr.interp_time(Q_([1, 2, 3], "s")).data,
    tsnew_expr.interp_time(Q_([1, 2, 3], "s")).data,
)

True